# Model Selection
In this notebook, the model selection capabilities of pyPESTO are demonstrated, which facilitate the selection of the best model from a set of possible models. This includes examples of forward, backward, and brute force methods, as well as criteria such as AIC, AICc, and BIC. Various additional options and convenience methods are also demonstrated.

All specification files use the [PEtab Select](https://github.com/PEtab-dev/petab_select) format, which is a model selection extension to the parameter estimation specification format [PEtab](https://github.com/PEtab-dev/PEtab).

Dependencies can be installed with `pip3 install pypesto[select]`.

## Example Model
This example involves a reaction system with two species (`A` and `B`), with their growth, conversion and decay rates controlled by three parameters ($\theta_1$, $\theta_2$, and $\theta_3$). Many different hypotheses will be considered, which are described in the model specifications file. There, a parameter fixed to zero indicates that the associated reaction(s) should not be in the model which resembles a hypothesis.

Synthetic measurement data is used here, which was generated with the "true" model. The comprehensive model includes additional behavior involving a third parameter ($\theta_3$). Hence, during model selection, models without $\theta_3=0$ should be preferred.

![](model_selection_network.jpeg)


## Model Space Specifications File

The model selection specification file can be written in the following compressed format.

| model_subspace_id | petab_yaml                             | $\theta_1$ | $\theta_2$ | $\theta_3$ |
|:---------|:----------------------------------|:----|:----|:----|
| M1      | example_modelSelection.yaml | 0;estimate  | 0;estimate  | 0;estimate  |

Alternatively, models can be explicitly specified. The below table is equivalent to the above table.

| model_subspace_id | petab_yaml                             | $\theta_1$ | $\theta_2$ | $\theta_3$ |
|:---------|:----------------------------------|:----|:----|:----|
| M1_0	| example_modelSelection.yaml	| 0	  | 0   |	0 |
| M1_1	| example_modelSelection.yaml	| 0	  | 0	| estimate |
| M1_2	| example_modelSelection.yaml	| 0	  | estimate |	0 |
| M1_3	| example_modelSelection.yaml	| estimate |	0	| 0 |
| M1_4	| example_modelSelection.yaml	| 0	  | estimate |	estimate |
| M1_5	| example_modelSelection.yaml	| estimate |	0   |	estimate |
| M1_6	| example_modelSelection.yaml	| estimate |	estimate |	0 |
| M1_7	| example_modelSelection.yaml	| estimate | estimate |	estimate |

Either of the above tables (as [TSV](https://en.wikipedia.org/wiki/Tab-separated_values) files) are valid inputs. Any combinations of cells in the compressed or explicit format is also acceptable, including the following example.

| model_subspace_id | petab_yaml                             | $\theta_1$ | $\theta_2$ | $\theta_3$ |
|:---------|:----------------------------------|:----|:----|:----|
| M1      | example_modelSelection.yaml | 0;estimate  | 0;estimate  | 0  |
| M2      | example_modelSelection.yaml | 0;estimate  | 0;estimate  | estimate  |

Due to the topology of the example model, setting $\theta_1$ to zero can result in a model with no dynamics. Hence, for this example, some parameters are set to non-zero fixed values. These parameters are considered as fixed (not estimated) values in criterion (e.g. AIC) calculations.

The model specification table used in this notebook is shown below.

In [ ]:
import pandas as pd
from IPython.display import HTML, display

df = pd.read_csv("model_selection/model_space.tsv", sep="\t")
display(HTML(df.to_html(index=False)))

## Forward Selection, Multiple Searches
Here, we show a typical workflow for model selection. First, a [PEtab Select](https://github.com/petab-dev/petab_select) problem is created, which is used to initialize a pyPESTO model selection problem.

In [ ]:
# Disable some logged messages that make the model selection log more
# difficult to read.
import tqdm


def nop(it, *a, **k):
    return it


tqdm.tqdm = nop

In [ ]:
import petab_select
from petab_select import ESTIMATE, Criterion, Method

import pypesto.select

petab_select_problem = petab_select.Problem.from_yaml(
    "model_selection/petab_select_problem.yaml"
)

In [ ]:
import logging

import pypesto.logging

pypesto.logging.log(level=logging.WARNING, name="pypesto.petab", console=True)

pypesto_select_problem_1 = pypesto.select.Problem(
    petab_select_problem=petab_select_problem
)

Models can be selected with a model selection algorithm (here: [forward](https://en.wikipedia.org/wiki/Stepwise_regression)) and a comparison criterion (here: [AIC](https://en.wikipedia.org/wiki/Akaike_information_criterion)). The forward method starts with the smallest model. Within each following iteration it tests all models with one additional estimated parameter.

To perform a single iteration, use `select` as shown below. Later in the notebook, `select_to_completion` is demonstrated, which performs multiple consecutive iterations automatically.

As no initial model is specified here, a virtual initial model with no estimated parameters is automatically used to find the "smallest" (in terms of number of estimated parameters) models. In this example, this is the model `M1_0`, which has no estimated parameters.

In [ ]:
# Reduce notebook runtime
model_problem_options = {
    "minimize_options": {
        "n_starts": 10,
        "filename": None,
        "progress_bar": False,
    },
}

best_model_1, _ = pypesto_select_problem_1.select(
    method=Method.FORWARD,
    criterion=Criterion.AIC,
    model_problem_options=model_problem_options,
)

To search more of the model space, hence models with more parameters, the algorithm can be repeated. As models with no estimated parameters have already been tested, subsequent `select` calls will begin with the next simplest model (in this case, models with exactly 1 estimated parameter, if they exist in the model space), and move on to more complex models.

The best model from the first iteration is supplied as the predecessor (initial) model here.

In [ ]:
best_model_2, _ = pypesto_select_problem_1.select(
    method=Method.FORWARD,
    criterion=Criterion.AIC,
    model_problem_options=model_problem_options,
    predecessor_model=best_model_1,
)

Plotting routines to visualize the best model at each iteration of the selection process, or to visualize the graph of models that have been visited in the model space are available.

In [ ]:
import petab_select.plot as plot

selected_models = [best_model_1, best_model_2]

plot_data = plot.PlotData(
    models=petab_select_problem.state.models,
    criterion=Criterion.AIC,
    relative_criterion=False,
)
plot.line_best_by_iteration(plot_data=plot_data)

plot_data.relative_criterion = True
plot.line_best_by_iteration(plot_data=plot_data);

In [ ]:
plot_data = plot.PlotData(
    models=petab_select_problem.state.models,
    criterion=Criterion.AIC,
)
plot_data.augment_labels(criterion=True)
plot.graph_history(plot_data=plot_data);

## Backward Selection, Custom Initial Model
Backward selection is specified by changing the algorithm from `Method.FORWARD` to `Method.BACKWARD` in the `select()` call.

A custom initial model is specified with the optional `predecessor_model` argument of `select()`.

In [ ]:
import numpy as np
from petab_select import Model

# The PEtab Select problem contains information about calibrated models,
# so that they are not recalibrated. In order to do a completely new
# model selection, one can reload the original PEtab Select problem.
petab_select_problem = petab_select.Problem.from_yaml(
    "model_selection/petab_select_problem.yaml"
)
pypesto_select_problem_2 = pypesto.select.Problem(
    petab_select_problem=petab_select_problem
)

model_subspace_petab_yaml = "model_selection/example_modelSelection.yaml"
initial_model = Model(
    model_id="myModel",
    model_subspace_petab_yaml=model_subspace_petab_yaml,
    model_subspace_id="dummy_myModel",
    model_subspace_indices=[0,0,0],
    parameters={
        "k1": 0.1,
        "k2": ESTIMATE,
        "k3": ESTIMATE,
    },
    criteria={petab_select_problem.criterion: np.inf},
)

print("Initial model:")
print(initial_model)

In [ ]:
pypesto_select_problem_2.select(
    method=Method.BACKWARD,
    criterion=Criterion.AIC,
    predecessor_model=initial_model,
    model_problem_options=model_problem_options,
);

In [ ]:
plot_data = plot.PlotData(
    models=petab_select_problem.state.models,
    criterion=Criterion.AIC,
)
plot_data.augment_labels(criterion=True)
plot.graph_history(plot_data=plot_data);

## Additional Options
There exist additional options that can be used to further customise selection algorithms.

### Select First Improvement
At each selection step, as soon as a model that improves on the previous model is encountered (by the specified criterion), it is selected and immediately used as the previous model in the next iteration of the selection. This is unlike the default behaviour, where all test models at each iteration are optimized, and the best of these is selected.

### Use Previous Maximum Likelihood Estimate as Startpoint
The maximum likelihood estimate from the previous model is used as one of the startpoints in the multistart optimization of the test models. The default behaviour is that all startpoints are automatically generated by pyPESTO.

### Minimize Options
Optimization can be customised with a dictionary that specifies values for the corresponding keyword arguments of [minimize](https://github.com/ICB-DCM/pyPESTO/blob/master/pypesto/optimize/optimize.py).

### Criterion Options
Currently implemented options are: `Criterion.AIC` ([Akaike information criterion](https://en.wikipedia.org/wiki/Akaike_information_criterion)), `Criterion.AICC` ([corrected AIC](https://en.wikipedia.org/wiki/Akaike_information_criterion)), and `Criterion.BIC` ([Bayesian information criterion](https://en.wikipedia.org/wiki/Bayesian_information_criterion)).

#### Criterion Threshold
A threshold can be specified, such that only models that improve on previous models by the threshold amount in the chosen criterion are accepted.

In [ ]:
petab_select_problem = petab_select.Problem.from_yaml(
    "model_selection/petab_select_problem.yaml"
)
pypesto_select_problem_3 = pypesto.select.Problem(
    petab_select_problem=petab_select_problem
)
best_models = pypesto_select_problem_3.select_to_completion(
    method=Method.FORWARD,
    criterion=Criterion.BIC,
    select_first_improvement=True,
    startpoint_latest_mle=True,
    model_problem_options=model_problem_options,
)

In [ ]:
plot_data = plot.PlotData(
    petab_select_problem.state.models,
    criterion=Criterion.BIC,
)
plot.line_best_by_iteration(plot_data=plot_data);

In [ ]:
plot_data.relative_criterion = False
plot_data.augment_labels(criterion=True)
plot.graph_history(plot_data=plot_data);

### Postprocessors

You can optionally provide postprocessing methods that operate on calibrated models. For example, there are postprocessors to save the pyPESTO result to disk, or save a summarized report of models to a TSV file. You can also write your own method, which should take only a single `pypesto.select.ModelProblem` object as input.

In [ ]:
# Repeat with AICc and criterion_threshold == 10
petab_select_problem = petab_select.Problem.from_yaml(
    "model_selection/petab_select_problem.yaml"
)
pypesto_select_problem_4 = pypesto.select.Problem(
    petab_select_problem=petab_select_problem
)

In [ ]:
# Setup a postprocessor that saves the pyPESTO result objects to disk, as well as a summarized report.
from functools import partial
from pypesto.select.postprocessors import save_postprocessor, report_postprocessor, multi_postprocessor
from pathlib import Path
import shutil
output_path = Path() / "output_select"
if output_path.exists():
    shutil.rmtree(str(output_path))
output_path.mkdir(exist_ok=True, parents=True)
model_postprocessor = partial(multi_postprocessor, postprocessors=[
    partial(save_postprocessor, output_path=output_path),
    partial(report_postprocessor, output_filepath=output_path / "report.tsv"),
])

best_models = pypesto_select_problem_4.select_to_completion(
    method=Method.FORWARD,
    criterion=Criterion.AICC,
    select_first_improvement=True,
    startpoint_latest_mle=True,
    model_problem_options=model_problem_options,
    criterion_threshold=10,
    model_postprocessor=model_postprocessor,
)

In [ ]:
df = pd.read_csv(output_path / "report.tsv", sep="\t")
df

In [ ]:
plot_data = plot.PlotData(
    models=petab_select_problem.state.models,
    criterion=Criterion.AICC,
)
plot.line_best_by_iteration(plot_data=plot_data);

In [ ]:
plot_data.relative_criterion = False
plot_data.augment_labels(criterion=True)
plot.graph_history(plot_data=plot_data);

### Saving results

Although pyPESTO results and summaries can be saved to disk via the postprocessors, most analysis methods will expect a `petab_select.Models` object, which is created during model selection and can also be saved to disk for later analysis.

In [ ]:
petab_select_problem.state.models.to_yaml(output_path / "my_models.yaml")
loaded_models = petab_select.Models.from_yaml(output_path / "my_models.yaml")

plot_data = plot.PlotData(
    models=loaded_models,
    criterion=Criterion.AICC,
    relative_criterion=False
)
plot_data.augment_labels(criterion=True)

### Additional plotting methods

There are additional plotting methods provided by the PEtab Select library, which are demonstrated in another notebook: https://petab-select.readthedocs.io/en/stable/examples/visualization.html

An example is this ordering of models according to the iteration in which they were calibrated. N.B. The predecessor of M1_7 is actually M1_1.

In [ ]:
plot.graph_iteration_layers(
    plot_data=plot_data,
    draw_networkx_kwargs={
        "arrowstyle": "-|>",
        "node_shape": "s",
        "node_size": 4000,
        "edgecolors": "k",
    },
);

## Multistart
Multiple model selections can be run by specifying multiple initial models.

In [ ]:
petab_select_problem = petab_select.Problem.from_yaml(
    "model_selection/petab_select_problem.yaml"
)
pypesto_select_problem_5 = pypesto.select.Problem(
    petab_select_problem=petab_select_problem
)

initial_model_1 = Model(
    model_id="myModel1",
    model_subspace_petab_yaml=model_subspace_petab_yaml,
    model_subspace_id="dummy_myModel1",
    model_subspace_indices=[0,0,0],
    parameters={
        "k1": 0,
        "k2": 0,
        "k3": 0,
    },
    criteria={petab_select_problem.criterion: np.inf},
)

initial_model_2 = Model(
    model_id="myModel2",
    model_subspace_petab_yaml=model_subspace_petab_yaml,
    model_subspace_id="dummy_myModel2",
    model_subspace_indices=[0,0,0],
    parameters={
        "k1": ESTIMATE,
        "k2": ESTIMATE,
        "k3": 0,
    },
    criteria={petab_select_problem.criterion: np.inf},
)

initial_models = [initial_model_1, initial_model_2]
best_model, best_models = pypesto_select_problem_5.multistart_select(
    method=Method.FORWARD,
    criterion=Criterion.AIC,
    predecessor_models=initial_models,
    model_problem_options=model_problem_options,
)

In [ ]:
plot_data = plot.PlotData(
    models=petab_select_problem.state.models,
    criterion=Criterion.AIC,
    relative_criterion=False,
)
plot.graph_history(plot_data);